# Capstone Project

## Importing libs

In [23]:
import pandas as pd
import numpy as np
import urllib.request

## Reading page 

In [24]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

## Finding table and cleaning data

In [34]:
# reading table to df

My_table=soup.find('table', class_='wikitable sortable')
A=[]
B=[]
C=[]
for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df=pd.DataFrame(A,columns=['Postalcode'])
df['Borough']=B
df['Neighborhood']=C
df = df.replace('\n','', regex=True)


# dropping not assigned borough and reseting index

index_del=df[df['Borough']=='Not assigned'].index
df.drop(index_del, inplace=True)
df = df.reset_index()
del df['index']


# assigning borough to empty neighborhood and making it comma sperated
df.loc[(df.Neighborhood == ''),'Neighborhood']=df.loc[(df.Neighborhood == ''),'Borough']
df = df.replace(' / ',', ', regex=True)

df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Printing Shape

In [27]:
df.shape

(103, 3)

# Reading Coordinates CSV and merging it with table

In [10]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

--2020-05-07 11:52:45--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 158.85.108.83, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-07 11:52:46--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-07 11:52:47--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-07 11:52:47--  https://ibm.box.com/publ

In [28]:
df1 = pd.read_csv('Geospatial_Coordinates.csv')
df1.rename(columns = {'Postal Code':'Postalcode'}, inplace = True)
df1.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df=pd.merge(df, df1, on=['Postalcode'])
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Printing Shape

In [36]:
df.shape

(103, 5)